# LangChain Fundamentals
***
## Table of Contents
***

## 1. Introduction

## 2. Environmental Variables
Firstly, environmental variables need to be configured. These variables, especially API keys should never be hardcoded or made visible to others. The [python-dotenv](https://pypi.org/project/python-dotenv/) libray makes it straightforward to securely access variables set in a `.env` file.

In [ ]:
import os
from getpass import getpass

try:
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    raise ImportError("Error: 'python-dotenv' not installed")

`os.environ` is a dictionary-like object representing the environment variables of the current process. It allows users to assign new values using the syntax: `os.environ['some_key'] = 'some_value'`

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)


## 3. Using Language Model
### Loading Model
There are multiple approaches to loading language models:
1. Use `init_chat_model` from `langchain.chat_models`, with specified model name and provider. 
2. Specify the model's integration package first, then call the appropriate method to initialise the model.

In [ ]:
# from langchain.chat_models import init_chat_model
from langchain_openai import ChatOpenAI


model_name = "gpt-4o-mini"

# 1. Simpler call
# model = init_chat_model(model=model_name, model_provider="openai", temperature=0.8)

# 2. Directly use of the integration package
model = ChatOpenAI(model=model_name, temperature=0.8)

### Interacting with Language Model
For a simple call, we can pass `messages` to the `.invoke` method. The list of message objects (`messages`) can be categorised into three parts:
- SystemMessage: Text that guides or determines AI's behaviour or actions.
- HumanMessage: Input given by a user.
- AIMessage: Output generated by the model.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Translate the following message from English into French"),
    HumanMessage(content="Today is a beautiful day"),
]

model.invoke(input=messages)

### Prompt Template
A prompt template provides a structured way of creating inputs for language models where parts of the prompt can be dynamically changed based on context or user input.

Prompts in LangChain can be split into three components:
- System Prompt: Gives instructions or a personality to the LLM model. This prompt determines the behaviour or characteristics of the model.
- User Prompt: Input given by a user.
- AI Prompt: Output generated by the model.

In [ ]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_prompt = SystemMessagePromptTemplate.from_template(
    template="You are an AI translater. Translate text from English to {language}",
    input_variables=["language"],
)

user_prompt = HumanMessagePromptTemplate.from_template(
    template="""Your task is to translate a text. The text to be translated is:
    ---
    {text}
    ---
    Output only the translated text, no other explanation or text should be provided
    """,
    input_variables=["text"],  # Define a variable
)

text_in_english = """
A croissant is a French Viennoiserie in a crescent shape made from a laminated yeast dough that sits between a bread and a puff pastry.
"""

target_language = "French"

Let's display the formatted user prompt after inserting a value into the `text` parameter:

In [ ]:
print(user_prompt.format(text=text_in_english))

After defining system and user prompts, we can merge them into a full chat prompt using `ChatPromptTemplate`:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

ChatPromptTemplate adds a prefix indicating a role of each message (e.g., System:, Human: or AI:).

In [ ]:
print(prompt_template.format(text=text_in_english, language=target_language))

Using **L**ang**C**hain **E**xpression **L**anguage (LCEL), we can construct a chain that links the user input, prompt templates, the model and the output in a sequence:

`input | prompt template | model | output`

This pipeline enables smooth data flow, where the user input is formatted by the prompt template, passed to the model for inference, and the model's response is captured as the output.

Note that lambda expressions are required to access prompt template variables. These variables are stored within input dictionaries or complex objects, thus the lambdas explicity extract or map the necessary fields from these data structures to ensure the correct values are passed to each stage in the chain.

In [ ]:
chain = (
    {"text": lambda x: x["text"], "language": lambda x: x["language"]}
    | prompt_template
    | model
    | {"translated_text": lambda x: x.content}
)

chain.invoke(input={"text": text_in_english, "language": target_language})

### Formatting with Pydantic
Using Pydantic, we can enforce a specific format or data structure on the output generated by the model.

In [17]:
from pydantic import BaseModel, Field


class Translation(BaseModel):
    org_text: str = Field(description="Original text")
    tl_text: str = Field(description="Translated text")
    n_words: int = Field(description="Number of words in the translated text")


structured_model = model.with_structured_output(Translation)

second_system_prompt = SystemMessagePromptTemplate.from_template(
    template="""
You are an AI translator. Translate a text from English to {language}.
""",
    input_variables=["language"],
)

second_user_prompt = HumanMessagePromptTemplate.from_template(
    template="""
    Your task is to translate the following text:
    ---
    {text}
    ---

    then count the number of words in the translated text. 
    Output only the translated text and the word counts, no other explanation or text should be provided.
""",
    input_variables=["text"],
)

second_prompt_template = ChatPromptTemplate.from_messages(
    [second_system_prompt, second_user_prompt]
)

second_chain = (
    {
        "text": lambda x: x["text"],
        "language": lambda x: x["language"],
    }
    | second_prompt_template
    | structured_model
    | {
        "org_text": lambda x: x.org_text,
        "tl_text": lambda x: x.tl_text,
        "n_words": lambda x: x.n_words,
    }
)
second_chain.invoke(input={"text": text_in_english, "language": target_language})

{'org_text': 'A croissant is a French Viennoiserie in a crescent shape made from a laminated yeast dough that sits between a bread and a puff pastry.',
 'tl_text': "Un croissant est une viennoiserie française en forme de croissant faite d'une pâte levée feuilletée qui se situe entre un pain et une pâte feuilletée.",
 'n_words': 25}

## 4. LangSmith
LangSmith is a comprehensive platform developed by the LangChain team for observability, debugging, testing, and evaluation of large language model (LLM) applications. It helps developers monitor and improve AI-powered apps by capturing detailed traces of interactions, including inputs, outputs, intermediate steps, execution times, and errors.

In [ ]:
os.environ["LANGSMITH_TRACING"] = "true"

os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY") or getpass(
    "Enter LangSmith API key: "
)
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT") or getpass(
    "Enter project name: "
)
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT") or getpass(
    "Enter LangSmith endpoint: "
)

print(f"Project name: {os.environ['LANGSMITH_PROJECT']}")

### Default Tracing

If the API keys and parameters were properly configured in the `.env` file, the executions above should have been traced on [LangSmith UI](https://eu.smith.langchain.com/). LangSmith automatically records logs (e.g., inputs, outputs, errors, and execution time) which greatly facilitates the debugging process.

![Default Trace](_images/default_trace.png)

### Non-LangChain Code Tracing
By adding the `@traceable` decorator, LangSmith will be able to trace non-LangChain functions.

In [ ]:
from langsmith import traceable
import random


@traceable
def generate_random_int():
    num = random.randint(1, 10)
    if num % 2 == 0:
        raise ValueError("Error: The value has to be odd.")
    else:
        return "Odd value. No problem."


generate_random_int()

'Odd value. No problem.'

![Traceable](_images/traceable.png)